#IMPORTING THE REQUIRED LIBRARIES
#creating the url and extracting the text from html page using BeautifulSoup library.

In [63]:
import pandas as pd
import numpy as np
import requests
print("libraries imported")

from urllib.request import urlopen
from bs4 import BeautifulSoup

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html=urlopen(url)

soup=BeautifulSoup(html,'lxml')

#extracting all the rows from the text.
rows=soup.find_all('tr')

#extracting all the cells and the values inside, from the rows.
for row in rows:
    row_td=soup.find_all('td')
    
#using Regular Expression(re) operation to clean the data.
import re
for row in rows:
    cells=row.find_all('td')
    str_cells=str(cells)
    clean=re.compile('<.*?>')
    clean2=(re.sub(clean,'',str_cells))

#coverting the data extracted into a dataframe.
df=pd.DataFrame(list_rows)
df.head()

libraries imported


,0
0,[]
1,"[M1A\n, Not assigned\n, Not assigned\n]"
2,"[M2A\n, Not assigned\n, Not assigned\n]"
3,"[M3A\n, North York\n, Parkwoods\n]"
4,"[M4A\n, North York\n, Victoria Village\n]"


#splitting the data into columns.

In [59]:
df1=df[0].str.split('\n,',expand=True)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,[M1A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[M2A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,[M3A,North York,Parkwoods\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,[M4A,North York,Victoria Village\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


#dropping unnecesary columns from the dataframe.

In [60]:
df2=df1.drop([0,181,182,183,184])
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
1,[M1A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[M2A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,[M3A,North York,Parkwoods\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,[M4A,North York,Victoria Village\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,[M5A,Downtown Toronto,"Regent Park, Harbourfront\n]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


#dropping unnecessary rows and cleaninf the dataframe further using strip.

In [61]:
df2=df2.drop(list(range(3,31)),axis=1)
df2[0]=df2[0].str.strip('[')
df2[2]=df2[2].str.strip('\n]')
df2.head()

,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#creating the headers for the dataframe and cleaning it.

In [14]:
col_labels=soup.find_all('th')
all_headers=[]
col_str=str(col_labels)
cleantext2=BeautifulSoup(col_str,'lxml').get_text()
cleantext2=cleantext2.strip('[')
cleantext2=cleantext2.strip('\n]')
cleantext2=cleantext2.split('\n,')
cleantext2

['Postal Code', ' Borough', ' Neighbourhood', ' Canadian postal codes']

#converting headers into required dataframe.

In [25]:
df3=pd.DataFrame([cleantext2])
df3.drop([3],axis=1,inplace=True)
df3

,0,1,2
0,Postal Code,Borough,Neighbourhood


#concating the header dataframe and the dataframe made out of the data extracted.

In [26]:
frames=[df3,df2]
df4=pd.concat(frames)
df4.head()

,0,1,2
0,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


#renaming the columns and dropping the unnecesary rows.

In [34]:
df5=df4.rename(columns=df4.iloc[0])
df5=df5.drop(df5.index[0])
df5.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#replacing the not assigned values with None.

In [38]:
df5.replace(' Not assigned',value=np.nan,inplace=True)
df5.head()

,Postal Code,Borough,Neighbourhood
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#Processing the cells that have an assigned borough. Ignoring cells with a borough that is Not assigned.

In [62]:
df5.dropna(axis=0,subset=[' Borough'],inplace=True)
df5.reset_index(drop=True,inplace=True)
df5.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#Finding the shape of the required final dataframe.

In [53]:
df5.shape

(103, 3)